In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

cancer_data = load_breast_cancer()

X_data = cancer_data.data 
y_label = cancer_data.target 

X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size=0.2, random_state=0)

In [3]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.linear_model import LogisticRegression 

knn_clf = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

lr_final = LogisticRegression(C=10)

In [4]:
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train, y_train)
dt_clf.fit(X_train, y_train)
ada_clf.fit(X_train, y_train)

,estimator,None
,n_estimators,100
,learning_rate,1.0
,algorithm,'deprecated'
,random_state,None


In [5]:
from sklearn.metrics import accuracy_score

knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print(f'KNN 정확도: {accuracy_score(y_test, knn_pred):.4f}')
print(f'랜덤 포레스트 정확도: {accuracy_score(y_test, rf_pred):.4f}')
print(f'결정 트리 정확도: {accuracy_score(y_test, dt_pred):.4f}')
print(f'AdaBoost 정확도: {accuracy_score(y_test, ada_pred):.4f}')







KNN 정확도: 0.9211
랜덤 포레스트 정확도: 0.9649
결정 트리 정확도: 0.9035
AdaBoost 정확도: 0.9737


In [6]:
import numpy as np

pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

pred = np.transpose(pred)
print(pred.shape)

(4, 114)
(114, 4)


In [7]:
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

print(accuracy_score(y_test, final))

0.9824561403508771


In [11]:
from sklearn.model_selection import KFold 

def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
    kf = KFold(n_splits=n_folds, shuffle=False)
    train_fold_pred = np.zeros((X_train_n.shape[0], 1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__, ' model 시작')

    for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        print(f"폴드 세트: {folder_counter + 1} / {n_folds}")
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]

        model.fit(X_tr, y_tr)
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1, 1)
        test_pred[:, folder_counter] = model.predict(X_test_n)

    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1, 1)

    return train_fold_pred, test_pred_mean

In [12]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test, 7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)




KNeighborsClassifier  model 시작
폴드 세트: 1 / 7
폴드 세트: 2 / 7
폴드 세트: 3 / 7
폴드 세트: 4 / 7
폴드 세트: 5 / 7
폴드 세트: 6 / 7
폴드 세트: 7 / 7
RandomForestClassifier  model 시작
폴드 세트: 1 / 7
폴드 세트: 2 / 7
폴드 세트: 3 / 7
폴드 세트: 4 / 7
폴드 세트: 5 / 7
폴드 세트: 6 / 7
폴드 세트: 7 / 7
DecisionTreeClassifier  model 시작
폴드 세트: 1 / 7
폴드 세트: 2 / 7
폴드 세트: 3 / 7
폴드 세트: 4 / 7
폴드 세트: 5 / 7
폴드 세트: 6 / 7
폴드 세트: 7 / 7
AdaBoostClassifier  model 시작
폴드 세트: 1 / 7
폴드 세트: 2 / 7
폴드 세트: 3 / 7
폴드 세트: 4 / 7
폴드 세트: 5 / 7
폴드 세트: 6 / 7
폴드 세트: 7 / 7


In [13]:
Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)

print(X_train.shape, X_test.shape)
print(Stack_final_X_train.shape, Stack_final_X_test.shape)



(455, 30) (114, 30)
(455, 4) (114, 4)


In [14]:
lr_final.fit(Stack_final_X_train, y_train)
stack_final = lr_final.predict(Stack_final_X_test)

print(accuracy_score(y_test, stack_final))






0.9736842105263158
